In [14]:
import copy
import os
import cobra
from tests import TEST_DIR

aragem_model_path = os.path.join(TEST_DIR, 'data', 'AraGEM2010.xml')

non_diel_model = cobra.io.read_sbml_model(aragem_model_path)

In [27]:
non_diel_model_copy = copy.deepcopy(non_diel_model)

In [21]:
non_diel_model_copy._compartments["c_Day"] = "Cytosol Day"

In [26]:
non_diel_model_copy.compartments

{'c': 'Cytosol',
 'p': 'Plastid',
 'm': 'Mitochondrion',
 'x': 'Peroxisome',
 'v': 'Vacuole',
 'biomass': 'Biomass',
 'acc': 'Accumulation',
 'ext': 'External'}

In [28]:
from cobra import Model
from typing import Dict

In [29]:
class CompartmentsCreation:

    def __init__(self, model: Model) -> None:
        """
        Constructor

        Parameters
        ----------
        model
        """
        self.model = model
        self.compartment_dict: Dict[str, str] = {"_c_Day": "c_Day", "_p_Day": "p_Day", "_m_Day": "m_Day", "_x_Day": "x_Day", "_v_Day": "v_Day", "_biomass_Day": "biomass_Day", "_acc_Day": "acc_Day", "_ext_Day": "ext_Day", "_c_Night": "c_Night", "_p_Night": "p_Night", "_m_Night": "m_Night", "_x_Night": "x_Night", "_v_Night": "v_Night", "_biomass_Night": "biomass_Night", "_acc_Night": "acc_Night", "_ext_Night": "ext_Night"}

    def day_attribution(self) -> Model:
        """
        Loop into the reactions and metabolites of a metabolic model
        Returns
        -------
        Reactions and metabolites ids with "_Day" as sufix
        """
        for r in self.model.reactions:
            r.id = r.id + "_Day"
        for m in self.model.metabolites:
            m.id = m.id + "_Day"

        return self.model.metabolites

    def duplicate(self) -> Model:
        """
        Function that duplicates the original reactions and replaces
        the suffix "_Day" with the suffix "_Night" and further duplicates
        the corresponding metabolites

        Returns
        -------
        The model with the reactions and the metabolites duplicated and properly identified
        """
        for r in self.model.reactions:
            duplicate_r = r.copy()
            duplicate_r.id = r.id.replace("_Day", "_Night")
            self.model.add_reactions([duplicate_r])
            for m in r.metabolites:
                duplicate_m = m.copy()
                duplicate_m.id = m.id.replace("_Day", "_Night")
                duplicate_r.add_metabolites({duplicate_m: r.metabolites[m]})

        return self.model

    def remove_day_met(self) -> None:
        """
        Function that identifies metabolites with the suffix "_Day" within
        reactions with the suffix "_Night"

        Returns
        -------
        Night reactions only with Night metabolites
        """
        for r in self.model.reactions:
            if r.id.endswith("_Night"):
                for m in r.metabolites:
                    if m.id.endswith("_Day"):
                        r.add_metabolites({m: -r.metabolites[m]})

    def compartments_division(self) -> Model:
        """
        Function that runs through the suffixes of each metabolite and
        assigns it to a compartment

        Returns
        -------
        Object model compartment with the Day and Night version for each
        of the original compartments
        """
        for m in self.model.metabolites:
            for k, v in self.compartment_dict.items():
                if m.id.endswith(k):
                    m.compartment = v
                    break

        return self.model.compartments

In [30]:
test = CompartmentsCreation(non_diel_model_copy)

In [31]:
test.day_attribution()

[<Metabolite S(43)45__40_3S_44_4R_41__45_cis_45_3_44_4_45_Dihydroxy_45_3_44_4_45_dihydrofluorene_c_Day at 0x7faa89ab2b20>,
 <Metabolite S(43)45_7_45_Isojasmonic_32_acid_32_CoA_c_Day at 0x7faa89ae6820>,
 <Metabolite S(45)45_Epiafzelechin_c_Day at 0x7faa89cd8d90>,
 <Metabolite S(45)45_Epicatechin_c_Day at 0x7faa89cd8eb0>,
 <Metabolite S(45)45_Epigallocatechin_c_Day at 0x7faa89cd8d30>,
 <Metabolite S(45)45_Ureidoglycolate_c_Day at 0x7faa89bd76a0>,
 <Metabolite S(45)45_alpha_45_Pinene_c_Day at 0x7faa89d62550>,
 <Metabolite S(45)45_beta_45_Pinene_c_Day at 0x7faa89d62a30>,
 <Metabolite S__40_11Z_44_14Z_41__45_Icosadienoyl_45_CoA_c_Day at 0x7faa89ba1640>,
 <Metabolite S__40_11Z_44_14Z_44_17Z_41__45_Icosatrienoyl_45_CoA_c_Day at 0x7faa89ba1940>,
 <Metabolite S__40_13Z_41__45_Docosenoic_32_acid_c_Day at 0x7faa89ba11f0>,
 <Metabolite S__40_13Z_44_16Z_41__45_Docosadi_45_13_44_16_45_enoyl_45_CoA_c_Day at 0x7faa89b3c070>,
 <Metabolite S__40_15S_41__45_15_45_Hydroxy_45_5_44_8_44_11_45_cis_45_13_45_t

In [32]:
test.duplicate()

Ignoring reaction 'Bc1_Night' since it already exists.
Ignoring reaction 'Bc2_Night' since it already exists.
Ignoring reaction 'Bc3_Night' since it already exists.
Ignoring reaction 'Bc4_Night' since it already exists.
Ignoring reaction 'Bc5_Night' since it already exists.
Ignoring reaction 'Bc6_Night' since it already exists.
Ignoring reaction 'Bc7_Night' since it already exists.
Ignoring reaction 'Bc8_Night' since it already exists.
Ignoring reaction 'Bc9_Night' since it already exists.
Ignoring reaction 'Bc10_Night' since it already exists.
Ignoring reaction 'Bc11_Night' since it already exists.
Ignoring reaction 'Bc12_Night' since it already exists.
Ignoring reaction 'Bc13_Night' since it already exists.
Ignoring reaction 'Bc14_Night' since it already exists.
Ignoring reaction 'Bc15_Night' since it already exists.
Ignoring reaction 'Bc16_Night' since it already exists.
Ignoring reaction 'Bc17_Night' since it already exists.
Ignoring reaction 'Bc18_Night' since it already exists.
I

Name,MODEL1507180028
Memory address,7faa8bced460
Number of metabolites,3466
Number of reactions,3202
Number of genes,1404
Number of groups,0
Objective expression,0.01*Ex16_Day - 0.01*Ex16_Day_reverse_efba9
Compartments,"Cytosol, Plastid, Mitochondrion, Peroxisome, Vacuole, Biomass, Accumulation, External"


In [7]:
test.remove_day_met()

In [8]:
test.compartments_division()

In [10]:
for reaction in non_diel_model_copy.reactions:
    print(f"Reaction: {reaction.id}")
    for metabolite in reaction.metabolites:
        print(f"Metabolite: {metabolite.id}")

#confirmar se bate certo as reações com os metabolitos correspondentes

Reaction: Bc1_Day
Metabolite: S_alpha_45_D_45_Glucose_c_Day
Metabolite: S_Glucose_biomass_Day
Reaction: Bc2_Day
Metabolite: S_Sucrose_c_Day
Metabolite: S_Sucrose_biomass_Day
Reaction: Bc3_Day
Metabolite: S_Cellulose_c_Day
Metabolite: S_Cellulose_biomass_Day
Reaction: Bc4_Day
Metabolite: S_Starch_p_Day
Metabolite: S_Starch_biomass_Day
Reaction: Bc5_Day
Metabolite: S_D_45_Xylose_c_Day
Metabolite: S_D_45_Xylose_biomass_Day
Reaction: Bc6_Day
Metabolite: S_L_45_Glutamyl_45_tRNA_40_Glu_41__c_Day
Metabolite: S_L_45_Glutamate_biomass_Day
Metabolite: S_tRNA_40_Glu_41__c_Day
Reaction: Bc7_Day
Metabolite: S_L_45_Alanyl_45_tRNA_c_Day
Metabolite: S_L_45_Alanine_biomass_Day
Metabolite: S_tRNA_40_Ala_41__c_Day
Reaction: Bc8_Day
Metabolite: S_L_45_Aspartyl_45_tRNA_40_Asn_41__c_Day
Metabolite: S_L_45_Aspartate_biomass_Day
Metabolite: S_tRNA_40_Asn_41__c_Day
Reaction: Bc9_Day
Metabolite: S_Glycyl_45_tRNA_40_Gly_41__c_Day
Metabolite: S_Glycine_biomass_Day
Metabolite: S_tRNA_40_Gly_41__c_Day
Reaction: Bc1

In [138]:
values = ['Cytosol Day', 'Plastid Day', 'Mitochondrion Day', 'Peroxisome Day', 'Vacuole Day', 'Biomass Day', 'Accumulation Day', 'External Day', 'Cytosol Night', 'Biomass Night', 'Plastid Night', 'Accumulation Night', 'External Night', 'Mitochondrion Night', 'Vacuole Night', "Peroxisome Night"]

#for idx, key in enumerate(non_diel_model_copy.compartments.keys()):
#    non_diel_model_copy.compartments[key] = values[idx]

In [142]:
non_diel_model_copy.compartments

{'c_Day': '',
 'p_Day': '',
 'm_Day': '',
 'x_Day': '',
 'v_Day': '',
 'biomass_Day': '',
 'acc_Day': '',
 'ext_Day': '',
 'c_Night': '',
 'biomass_Night': '',
 'p_Night': '',
 'acc_Night': '',
 'ext_Night': '',
 'm_Night': '',
 'v_Night': '',
 'x_Night': ''}

In [11]:
cobra.io.write_sbml_model(non_diel_model_copy, "AraGEM2010_copy.xml")